# Multi Label Text Classification using TensorFlow and 🔭 Galileo

In this tutorial, we'll train a model with Tensorflow and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality` with `pip install dataquality`

try:
    import dataquality as dq
except ImportError:
    # Upgrade pip
    !pip install -U pip &> /dev/null

    # Install HF datasets for downloading the example datasets
    !pip install -U dataquality datasets tensorflow &> /dev/null
    
    print('👋 Installed necessary libraries.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')

# 1. Login to Galileo

In [ ]:
import dataquality as dq

dq.login()

# 2. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can find more datasets [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:text-classification&task_ids=task_ids:multi-label-classification&sort=downloads).

dataset_name = 'go_emotions' #@param ["go_emotions", "lex_glue"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)

# Check that the dataset has at least train and either of validation/test
assert "train" in data and {"validation", "test"}.intersection(data), \
f"💾 The dataset {dataset_name} has either no train, or no validation or test splits, select another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

In [ ]:
#@markdown Convert HF dataset to Pandas dataframes 
import pandas as pd
import numpy as np

def load_pandas_df(data):
  # Find the name of the ground truth column
  good_col_names = [name for name in list(data['train'].features) if "label" in name]
  if len(good_col_names) == 1:
    label_col = good_col_names[0]
  else:
    col_names = list(data['train'].features)
    print(f"The name of the columns are {col_names}.")
    label_col = input(f"🏅 Please enter the name of the column containing the labels: ")
    assert label_col in col_names, f"{label_col} is not an existing column"

  # Load the labels in a dictionary
  num_classes = len(data['train'].features[label_col].feature.names)
  labels_cols = data['train'].features[label_col].feature.int2str(range(0, num_classes))

  def binarize_label_indices(label_idxs):
    a = np.zeros(len(labels_cols), dtype=int)
    a[label_idxs] = 1
    return a

  # Load the train data into a frame
  train_data = data["train"]
  train_df = pd.DataFrame.from_dict(train_data)
  train_labels = list(map(binarize_label_indices, data['train'][label_col]))
  _train_df_labels = pd.DataFrame(train_labels, columns=labels_cols)
  train_df = pd.concat([train_df, _train_df_labels], axis=1)
  train_df['id'] = train_df.index

  # Load the test data into a frame
  test_split_name = "validation" if "validation" in data else "test"
  test_data = data[test_split_name]
  test_df = pd.DataFrame.from_dict(test_data)
  test_labels = list(map(binarize_label_indices, data[test_split_name][label_col]))
  _test_df_labels = pd.DataFrame(test_labels, columns=labels_cols)
  test_df = pd.concat([test_df, _test_df_labels], axis=1)
  test_df['id'] = test_df.index
  
  return train_df, test_df, labels_cols

train_df, test_df, labels_cols = load_pandas_df(data)

if _minimize_for_ci():
  train_df, test_df = train_df[:10], test_df[:10]

# 3. Initialize Galileo

In [ ]:
# 🔭🌕 Galileo logging
dq.init(task_type="text_multi_label", 
        project_name="multi_label_text_classification_tensorflow", 
        run_name=f"example_run_{dataset_name.replace('/', '-')}")

# 4. Log input data with Galileo
Input data can be logged via `log_data_samples` (or `log_dataset` for logging iterables). This step will log input samples, gold labels, data split, and list of all labels. You can achieve this by adding 1 line of code to the standard PyTorch Dataset Class.

In [ ]:
from transformers import AutoTokenizer
import tensorflow as tf

# 🔭🌕 Galileo logging
dq.set_tasks_for_run(labels_cols, binary=True)
dq.log_data_samples(texts=train_df["text"], 
                    task_labels=train_df[labels_cols].apply(lambda row: list(row[row == 1].index.values), axis=1),
                    ids=train_df["id"],
                    split="training")
dq.log_data_samples(texts=test_df["text"], 
                    task_labels=test_df[labels_cols].apply(lambda row: list(row[row == 1].index.values), axis=1),
                    ids=test_df["id"],
                    split="test")


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
BATCH_SIZE = 16
# Train and test datasets
datasets = []
for df in [train_df, test_df]:
  inputs = tokenizer(df.text.to_list(), truncation=True, padding=True)
  inputs["label"] = df[labels_cols]
  inputs["id"] = df["id"]
  dataset = tf.data.Dataset.from_tensor_slices(dict(inputs)).batch(BATCH_SIZE)
  datasets.append(dataset)

train_ds, test_ds = datasets

# 5. Log model data with Galileo

Model data can be logged via `log_model_outputs`. This step will log the model logits and embeddings. You can achieve this by adding 1 line of code to the standard pytorch model. 

We log [CLS]-token embedding from final layer, but you can log any custom layer for embeddings. 

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel

class TextMultiLabelClassificationModel(tf.keras.Model):
  def __init__(self, num_tasks):
    super().__init__()
    self.feature_extractor = TFAutoModel.from_pretrained("distilbert-base-uncased")

    # Check into this hidden size
    hidden_size = self.feature_extractor.config.hidden_size
    smaller_hidden_size = num_tasks + bool(hidden_size >= num_tasks)*(hidden_size - num_tasks) // 4
    self.pre_classifier = tf.keras.layers.Dense(smaller_hidden_size)
    self.classifier = tf.keras.layers.Dense(num_tasks)

  def call(self, x, ids):
    input_ids, attention_mask = x
    """Model forward function."""
    encoded_layers = self.feature_extractor(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    classification_embedding = encoded_layers[:, 0] # Extract [CLS]-token

    emb = self.pre_classifier(classification_embedding)
    emb = tf.nn.dropout(tf.nn.relu(emb), rate=0.1)
    logits = self.classifier(emb)

    # 🔭🌕 Galileo logging
    dq.log_model_outputs(
        embs=classification_embedding, logits=logits, ids=ids
    )

    return logits

# 6. Putting into Action: Training a Model

We complete the training pipeline by using a standard PyTorch training setup. While training, we log the current `epoch` and `split`. To complete logging, we call `dq.finish()` after training.

In [ ]:
from tqdm.notebook import tqdm

NUM_EPOCHS = 2

model = TextMultiLabelClassificationModel(len(labels_cols))
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Sparse means label can be provided as class idx, not one hot
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Iterate over epochs.
for epoch in range(NUM_EPOCHS):
  print("Start of epoch %d" % (epoch,))
  dq.set_epoch(epoch) # 🔭🌕 Galileo logging

  # Iterate over the batches of the train dataset.
  dq.set_split("training") # 🔭🌕 Galileo logging
  for train_batch in tqdm(train_ds):
      y = train_batch["label"]
      x = (train_batch["input_ids"], train_batch["attention_mask"])

      with tf.GradientTape() as tape:
          logits = model(
              x,
              ids=train_batch["id"],
          )
          loss = loss_fn(y, logits)

      grads = tape.gradient(loss, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))


  # Iterate over the batches of the test dataset.
  dq.set_split("test") # 🔭🌕 Galileo logging
  for test_batch in tqdm(test_ds):
      y = test_batch["label"]
      x = (test_batch["input_ids"], test_batch["attention_mask"])

      logits = model(x, ids=test_batch["id"])
      loss = loss_fn(y, logits)

dq.finish() # 🔭🌕 Galileo logging

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)